In [2]:
import pandas as pd
import glob
import os
path = "data/"
all_files = glob.glob(os.path.join(path , "*.csv"))

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

df_kick = pd.concat(li, axis=0, ignore_index=True)


In [3]:
df_kick.describe()

,backers_count,blurb,category,converted_pledged_amount,country,created_at,creator,currency,currency_symbol,currency_trailing_code,...,slug,source_url,spotlight,staff_pick,state,state_changed_at,static_usd_rate,urls,usd_pledged,usd_type
count,418499,418483,418499,418499,418499,418499,418499,418499,418499,418499,...,418499,418499,418499,418499,418499,418499,418499.0,418499,418499.0,417539
unique,6493,180701,170,62775,23,364317,208563,15,7,7,...,182265,170,7,7,6,344097,22391.0,182555,160005.0,3
top,0,Help support the largest creative collaboratio...,"{""id"":28,""name"":""Product Design"",""slug"":""desig...",0,US,created_at,creator,USD,$,True,...,slug,https://www.kickstarter.com/discover/categorie...,True,False,successful,1530676740,1.0,urls,0.0,international
freq,18362,104,8702,19242,296978,55,55,296978,336518,171111,...,55,4800,117465,182231,234930,56,148490.0,55,18362.0,209016


In [10]:
df_kick.usd_type.unique()

array(['international', 'domestic', nan, 'usd_type'], dtype=object)